# install librarys

In [ ]:
!pip install -U openai
!pip install -U langchain
!pip install -U langchain_openai
!pip install -U pydantic
!pip install -U langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


# connect to OpenAI gpt API

**To connect to the OpenAI GPT API, we utilized [AvalAi](https://avalai.ir/). After signing in, we generated an API key specifically for our project.**


In [ ]:
from langchain_openai import ChatOpenAI,OpenAI # pip install -U langchain_openai
from langchain_community.callbacks import get_openai_callback

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello world!"},
]
model_name = "gpt-4o-mini" # in this case we want to use gpt-4o-mini


llm = ChatOpenAI(
    model=model_name,
    base_url="https://api.avalai.ir/v1",
    temperature=None,
    max_tokens=100, #token limiter
    timeout=None,
    max_retries=0,



    api_key="aa-***"
)
# this is testing the API connection and tracking token usage
with get_openai_callback() as cb:
  response = llm.invoke(messages)
  print(cb)



Tokens Used: 30
	Prompt Tokens: 20
		Prompt Tokens Cached: 0
	Completion Tokens: 10
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $8.999999999999999e-06


## Before the trainning part we want to make a class for cleaner output using pydantic

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Dict, Union

class SentimentAnalysisResult(BaseModel):
    Overall_sentiment: str = Field(description="basic emotion of comment", alias="Overall sentiment")
    Inner_emotion: List[str] = Field(description="deeply emotions of user", alias="Inner emotion")
    Recommend: Union[str, bool] = Field(alias="Recommend")
    Mentioned_Aspects: List[str] = Field(description="Specific product features or components mentioned in the comment", alias="Mentioned Aspects")
    Aspect_Opinions: Dict[str, str] = Field(description="User's sentiment toward each mentioned product aspect", alias="Aspect Opinions")

    class Config:
        populate_by_name = True

## Train and test
for this section you can fine train comment in online shops like amazone digikala alibaba ...
i used digikalas comments


In [ ]:
import json
from pydantic import ValidationError

def analyze_comment():
  comment = input("enter your comment: ")
  messages = [
    {"role": "system","content":
    "You are a professional sentiment analyst. Your task is to emotionally analyze user comments (without considering the context) "
    "and identify whether the comment recommends buying the product or not.\n\n"
    "Your output must include **five parts**:\n\n"
    "1. **Overall Sentiment**: One of the following categories: positive, leaning positive, neutral, leaning negative, negative.\n"
    "2. **Dominant Emotions**: Identify the user's primary inner emotions based on the following categories: "
    "Anger, Contempt, Disgust, Enjoyment, Fear, Sadness, Surprise. You can list more than one emotion.\n"
    "3. **Recommendation**: Decide whether the comment recommends the product. Answer with true, false, maybe.\n"
    "4. **Mentioned Aspects**: List product aspects the user mentioned.\n"
    "5. **Aspect Opinions**: A dictionary where the keys are the mentioned product aspects and the values are the user's sentiment toward them ('good', 'bad', 'excellent', 'poor')\n\n"
    "Please respond strictly in this JSON format:\n"
    "{\n"
    '  "Overall sentiment": "Neutral",\n'
    '  "Inner emotion": ["Surprise"],\n'
    '  "Recommend": "maybe",\n'
    '  "Mentioned Aspects": ["battery", "screen"],\n'
    '  "Aspect Opinions": {\n'
    '  "battery": "good",\n'
    '  "screen": "average"\n'
    '  }\n'
    "}"
},





     #Positive samples

    {"role": "user", "content": "With my normal use, it holds a charge for 2 days. It may feel heavy in your hand at first. It doesn't feel like a low-quality phone at all. The first time I turned it on, a security update came, and then the Android 15 and One UI 7 updates immediately followed. I don't play games with it, but in my normal use, the phone is good and smooth. And one thing is that when the phone screen is off, if you get a message, no message appears on the screen, meaning the screen doesn't turn on at all. I don't know why it doesn't have Always On Display. You will get Gemini's AI feature with the Android 15 update. Overall, I'm satisfied with this phone for the price I paid."},
    {"role": "assistant", "content": '{"Overall sentiment": "Positive", "Inner emotion": ["Enjoyment", "Sadness", "Surprise"], "Recommend": "true", "Mentioned Aspects": ["battery life", "weight", "price", "build quality"], "Aspect Opinions": {"battery life": "good","weight": "average","build quality": "good","price": "good" } }'},

    {"role": "user", "content": "In my opinion, you shouldn't skip the A16, and buy the A15. I liked the A16 better and it has fewer problems than the previous model. Only for those who use a smartphone with a 3.5mm jack, the A15 is better and a little smaller. But the A16 is much smoother and has a better camera and a larger screen for watching videos, and it is dust and water resistant, and I had more fun with this phone. I recommend getting the A16."},
    {"role": "assistant", "content": '{"Overall sentiment": "Positive", "Inner emotion": ["Enjoyment", "Surprise"], "Recommend": "true", "Mentioned Aspects": ["camera", "screen", "dust and water resistance", "size"], "Aspect Opinions": {"camera": "good", "screen": "good", "dust and water resistance": "good", "size": "average"}}'},

    {"role": "user", "content": "I think it's a very good phone. I bought it on installments. In my opinion, it can compete with expensive phones."},
    {"role": "assistant", "content": '{"Overall sentiment": "Positive", "Inner emotion": ["Enjoyment"], "Recommend": "true", "Mentioned Aspects": ["price", "installments", "performance"], "Aspect Opinions": {"price": "good", "installments": "good", "performance": "good"}}'},

    {"role": "user", "content": "I bought the A16 phone from the Digikala in-person store on the ground floor of the Charso Passage! Incredibly, every single Chaarso seller offered it cheaper, and even 200 Tomans cheaper from the Digikala online store itself (11,500,000 purchased on 12/04). I suggest you go and buy it in person! In terms of your expectations from this mobile, I must say that you don't expect much from the phone! Who is this phone good for? Someone who needs a second phone will use it with their bank and other Iranian apps! Its battery lasts about 10 hours (calls, Iranian apps, web browsing, and a little Insta). Its camera is terrible and don't expect anything special from it! The screen is good and doesn't bother you at all. One UI 6.1 and Android 14 are installed, something that Samsung has promised will get 6 years of updates! I emphasize that if you expect anything from this phone like me, you will definitely be satisfied with it and it will work for you for a few years. Overall, it is a good and trouble-free phone if you don't expect the S25 Ultra from it!"},
    {"role": "assistant", "content": '{"Overall sentiment": "Positive", "Inner emotion": ["Enjoyment", "Surprise"], "Recommend": "true", "Mentioned Aspects": ["price", "battery", "camera", "screen", "software", "updates", "performance"], "Aspect Opinions": {"price": "good", "battery": "average", "camera": "poor", "screen": "good", "software": "good", "updates": "good", "performance": "good"}}'},


    #Leaning Positive samples

    {"role": "user", "content": "I received it today, the 31st of April. When I tried it, it had a charging cable but no cable. At first, I saw the satisfaction, then I ordered it. It seems like a good phone, but I wanted to get 256GB of memory, but my budget didn't meet it. Thank you, Digi Kala."},
    {"role": "assistant", "content": '{"Overall sentiment": "Leaning positive", "Inner emotion": ["Surprise", "Enjoyment"], "Recommend": "maybe", "Mentioned Aspects": ["charging cable", "memory", "price"], "Aspect Opinions": {"charging cable": "poor", "memory": "average", "price": "average"}}'},

    {"role": "user", "content": "It is quite satisfactory for the price. Only the removal of the 3.5 jack and the shared memory space with the SIM card is a bit of a bummer, which seems to be happening in all phones and you have to get used to it. The camera quality is good in daylight. The speaker is loud and its quality is also good. The battery also holds a charge well. The only thing I am unhappy with is the packaging of Digi, which does not include the phone in the box."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning positive", "Inner emotion": ["Enjoyment", "Sadness"], "Recommend": "true", "Mentioned Aspects": ["price", "3.5 jack", "memory space", "camera quality", "speaker", "battery", "packaging"], "Aspect Opinions": {"price": "good", "3.5 jack": "poor", "memory space": "poor", "camera quality": "good", "speaker": "good", "battery": "good", "packaging": "poor"}}'},

    {"role": "user", "content": "It was supposed to come with a charger, but it didn't come with it, but it arrived in good condition and well packaged."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning positive", "Inner emotion": ["Sadness", "Surprise"], "Recommend": "maybe", "Mentioned Aspects": ["charger", "packaging", "condition"], "Aspect Opinions": {"charger": "poor", "packaging": "good", "condition": "good"}}'},

    {"role": "user", "content": "Up-to-date and with many update capabilities, the screen was of very good quality, but you can't count on the camera too much."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning positive", "Inner emotion": ["Enjoyment", "Sadness"], "Recommend": "true", "Mentioned Aspects": ["updates", "screen", "camera"], "Aspect Opinions": {"updates": "good", "screen": "good", "camera": "poor"}}'},

    {"role": "user", "content": "It's good, but it has 2 problems: it doesn't have a headphone jack and the speaker quality is poor, even compared to its previous generations. The most important feature of this phone is its high-quality screen, the rest is average. I would give it 5 stars if the screen was my priority. Have a good purchase (-:"},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning positive", "Inner emotion": ["Enjoyment", "Disappointment"], "Recommend": "maybe", "Mentioned Aspects": ["headphone jack", "speaker", "screen"], "Aspect Opinions": {"headphone jack": "poor", "speaker": "poor", "screen": "good"}}'},


    #Neutral samples

    {"role": "user", "content": "Friends, according to the reviews I had this week, the phone's screen had a green tint problem in dark theme mode, and you only notice it when the backlight is low and in dark theme, open the settings menu and you will notice that on the two side edges, and of course for me it was mostly the left edge, in these areas the green and bright color dominates the black light! And the screen is not solid black, test a gray screen to test!! If you encounter something like this, you should contact the warranty!! I will send some pictures to understand the situation! The dark theme must be dark and the brightness must be at 10% for the test!!"},
    {"role": "assistant", "content": '{ "Overall sentiment": "Neutral", "Inner emotion": ["Surprise", "Fear"], "Recommend": "false", "Mentioned Aspects": ["screen", "dark theme", "brightness", "warranty"], "Aspect Opinions": {"screen": "poor", "dark theme": "poor", "brightness": "average", "warranty": "good"}}'},

    {"role": "user", "content": "The phone was sealed, so there's nothing special about it that I want to say, whether it was positive or negative. I can't say anything about a new, sealed phone, but I got it as quickly as possible."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Neutral", "Inner emotion": ["Surprise"], "Recommend": "maybe", "Mentioned Aspects": ["packaging", "delivery"], "Aspect Opinions": {"packaging": "good", "delivery": "good"}}'},

    {"role": "user", "content": "I haven't used it much yet, so I can't judge its performance. The design seems fine, but time will tell how it holds up."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Neutral", "Inner emotion": ["Surprise", "Fear"], "Recommend": "maybe", "Mentioned Aspects": ["performance", "design"], "Aspect Opinions": {"performance": "unknown", "design": "good"}}'},

    {"role": "user", "content": "The device comes with the standard accessories mentioned in the listing. The display size and weight are within normal range for phones in this category. I will need more usage before forming a detailed opinion."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Neutral", "Inner emotion": ["Surprise"], "Recommend": "true", "Mentioned Aspects": ["accessories", "display size", "weight"], "Aspect Opinions": {"accessories": "good", "display size": "average", "weight": "average"} }'},

    {"role": "user", "content": "I got the phone yesterday. Setup was easy. Calls, messages, and Wi-Fi work fine. I haven’t tested the battery or speed much yet."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Neutral", "Inner emotion": ["Surprise"], "Recommend": "true", "Mentioned Aspects": ["setup", "calls", "Wi-Fi", "battery", "speed"], "Aspect Opinions": {"setup": "good", "calls": "good", "Wi-Fi": "good", "battery": "unknown", "speed": "unknown"}}'},


     #Leaning Negative samples

    {"role": "user", "content": "I bought it for my mother, I also have an A54. There is no comparison, but it is larger than my phone in terms of accuracy, the screen bezels are more annoying, which is normal at this price. The noticeable thing for me is the lower quality of the screen compared to my phone, although both are AMOLED and thank God Samsung has abandoned LCD, but its colors and brightness are lower than the A54, which at first glance was a bit disappointing and I wish Samsung had spent a little more. Overall, the phone is acceptable."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning negative", "Inner emotion": ["Sadness", "Contempt"], "Recommend": "maybe", "Mentioned Aspects": ["screen", "bezels", "color quality", "brightness", "price"], "Aspect Opinions": {"screen": "average", "bezels": "poor", "color quality": "poor", "brightness": "poor", "price": "good"}}'},

    {"role": "user", "content": "The selfie and main camera are only acceptable in daylight and relatively bright light, it does not have OTG and does not have 4G+ internet, the battery is good, sometimes there is a delay in the user interface, it does not have many of the software features of the higher series of Samsung, such as Circle Search and artificial intelligence. At the current price of 13 million, it is not worth buying, and at the price of 18 million, there is a more reasonable option. Overall, I think you should buy the Samsung AC series and above, good luck"},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning negative", "Inner emotion": ["Sadness", "Disgust", "Anger"], "Recommend": "false", "Mentioned Aspects": ["camera", "OTG", "4G+", "battery", "user interface", "software features", "price"], "Aspect Opinions": {"camera": "average", "OTG": "poor", "battery": "good", "user interface": "poor", "software features": "poor", "price": "poor"}}'},

    {"role": "user", "content": "It's not bad, the sound quality is very average, the sound is low, I think you should spend more and get a better one, it's worth the price, not more, just the charging is good, the call quality is not good at all, the ambient sound is heard by the other party, I don't really recommend it."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning negative", "Inner emotion": ["Disgust", "Sadness"], "Recommend": "false", "Mentioned Aspects": ["sound quality", "charging", "call quality", "ambient sound"], "Aspect Opinions": {"sound quality": "average", "charging": "good", "call quality": "poor", "ambient sound": "poor"}}'},

    {"role": "user", "content": "After a while, his voice became quieter, it wasn't like before. Someone else had said the same thing. I said it must have been bad luck, until it happened to me."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning negative", "Inner emotion": ["Sadness"], "Recommend": "false", "Mentioned Aspects": ["voice quality"], "Aspect Opinions": {"voice quality": "poor"}}'},

    {"role": "user", "content": "Hello, there is no charging indicator. The sound quality is good for the price. The app is good, but contrary to friends' comments, it falls out of the ear and this is very annoying. In comparison, the HiLo W1 might be a better option."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Leaning negative", "Inner emotion": ["Sadness", "Disgust"], "Recommend": "false", "Mentioned Aspects": ["charging indicator", "sound quality", "app", "fit", "comparison"], "Aspect Opinions": {"charging indicator": "poor", "sound quality": "good", "app": "good", "fit": "poor", "comparison": "average"}}'},


    #Negative samples

    {"role": "user", "content": "The phone wouldn't register. Support said it would take a month. I called the dealer and he said return it, it's illegal. I bought it from Digikala recently and it says I'm not responsible for the registry for 30 days. Keep in mind the stress of the 30-day registry."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Negative", "Inner emotion": ["Anger", "Sadness", "Fear"], "Recommend": "false", "Mentioned Aspects": ["registration process", "support", "dealer", "purchase"], "Aspect Opinions": {"registration process": "poor", "support": "poor", "dealer": "poor", "purchase": "average"}}'},

    {"role": "user", "content": "I haven't had it for a year yet, the battery is still broken, protruding from the back."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Negative", "Inner emotion": ["Sadness", "Fear"], "Recommend": "false", "Mentioned Aspects": ["battery", "device condition"], "Aspect Opinions": {"battery": "poor", "device condition": "poor"}}'},

    {"role": "user", "content": "When the phone arrived and we tried to activate it, unfortunately, everything we did didn't work. We even took it to the warranty company and they said it had a software problem and we returned it and asked for another one to be sent. Digikala's technical support called and said they had set it up and since it was a new phone, the warranty company couldn't activate it either! They didn't go through with it and said they would send it again, which I suspect the phone wasn't flashed! 11 days have passed since they said they would send it again and I've called 1000 times and they keep saying they will follow up and still nothing! I'm really sorry to the Digikala team for this lack of coordination."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Negative", "Inner emotion": ["Anger", "Sadness", "Contempt"], "Recommend": "false", "Mentioned Aspects": ["activation", "warranty", "technical support", "coordination"], "Aspect Opinions": {"activation": "poor", "warranty": "poor", "technical support": "poor", "coordination": "poor"}}'},

    {"role": "user", "content": "Hello, 10 days after the purchase, the importing company has still not announced the phone's serial number to the peer system, and this is a big bug in itself. The device is under warranty with Aban DJ Company. I also contacted the company, but they are not responding. If you have this problem and your phone cannot be activated, please return it."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Negative", "Inner emotion": ["Anger", "Sadness", "Fear"], "Recommend": "false", "Mentioned Aspects": ["importing company", "serial number", "warranty", "customer service", "activation"], "Aspect Opinions": {"importing company": "poor", "serial number": "poor", "warranty": "good", "customer service": "poor", "activation": "poor"}}'},

    {"role": "user", "content": "I ordered a dual SIM phone and they sent me one SIM card."},
    {"role": "assistant", "content": '{ "Overall sentiment": "Negative", "Inner emotion": ["Anger", "Sadness"], "Recommend": "false", "Mentioned Aspects": ["SIM card", "order accuracy"], "Aspect Opinions": {"SIM card": "poor", "order accuracy": "poor"}}'},

    # test
    {"role": "user", "content": comment}
  ]
  with get_openai_callback() as cb:
    response = llm.invoke(messages)
    print(cb) # Move print outside try/except

  try:
    cleaned = response.content.strip()  # Remove leading/trailing spaces
    data = json.loads(cleaned)          # Parse the JSON string
    result = SentimentAnalysisResult(**data)  # Validate and convert to Pydantic model

    # Print in a nice format using original key names
    print(result.model_dump(by_alias=True))
  except ValidationError as e: # Catch specific Pydantic validation errors
    print("The model output doesn't match the expected format!")
    print("Pydantic Validation Error:", e)
    print("Raw output:", response.content)
  except Exception as e: # Catch other potential errors
    print("An unexpected error occurred!")
    print("Error:", e)
    print("Raw output:", response.content)

In [ ]:
analyze_comment()

enter your comment: Friends, according to the reviews I had this week, the phone's screen had a green tint problem in dark theme mode, and you only notice it when the backlight is low and in dark theme, open the settings menu and you will notice that on the two side edges, and of course for me it was mostly the left edge, in these areas the green and bright color dominates the black light! And the screen is not solid black, test a gray screen to test!! If you encounter something like this, you should contact the warranty!! I will send some pictures to understand the situation! The dark theme must be dark and the brightness must be at 10% for the test!!
Tokens Used: 4259
	Prompt Tokens: 4183
		Prompt Tokens Cached: 0
	Completion Tokens: 76
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0006730499999999999
{'Overall sentiment': 'Neutral', 'Inner emotion': ['Surprise', 'Fear'], 'Recommend': 'false', 'Mentioned Aspects': ['screen', 'dark theme', 'brightness', 'warranty']